## 1. Imports and Setup

In [1]:
from pathlib import Path
import sys
import os

project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

In [ ]:
# Import required libraries
import pandas as pd
import os
import omegaconf
from easyllm_kit.utils.io_utils import write_to_database
from easyllm_kit.utils import get_logger, read_json
from easyllm_kit.models import LLM
from easyllm_kit.configs.llm_base_config import GenerationArguments
# from famma_runner.runners.base_runner import Runner
from famma_runner.utils import (
    collect_images_from_first_subquestion, 
    generate_response_from_llm, 
    safe_parse_response,
    QuestionPrompt, 
    LANGUAGE_ORDER, 
    DC, 
    order_by_language, 
    ProgramOfThoughtsQuestionPrompt
)

# Initialize logger
logger = get_logger('generation_runner', 'generation_runner.log')

## 2. Configuration Management

In [3]:
from omegaconf import OmegaConf

In [ ]:
config_dir = Path.joinpath(project_root, 'configs', 'gen_config_qwen.yaml')
config = OmegaConf.load(config_dir)
config

## 3. Dataset Processing

In [5]:
# Load and process dataset
def load_dataset(data_path):
    """Load dataset from JSON file and prepare it for processing."""
    data = read_json(data_path)
    dataset_df = pd.DataFrame(data)
    
    # Create a new column for sorting languages
    order_by_language(dataset_df, LANGUAGE_ORDER, DC.MAIN_QUESTION_ID, DC.SUB_QUESTION_ID, DC.LANGUAGE)
    
    return dataset_df

In [6]:
# Filter dataset by question IDs
def filter_dataset_by_question_id(dataset_df, question_ids):
    """Filter dataset by specific question_ids."""
    filtered_main_question_ids = None
    
    if not question_ids:
        return dataset_df, filtered_main_question_ids
    
    # Convert single question_id to list for consistent processing
    if not isinstance(question_ids, omegaconf.ListConfig) and not isinstance(question_ids, list):
        question_ids = [question_ids]
    
    if len(question_ids) == 0:
        return dataset_df, filtered_main_question_ids
    
    # Create empty DataFrame to collect all filtered results
    filtered_results = pd.DataFrame()
    
    # Track unique language-main_question_id pairs to avoid duplicate processing
    processed_pairs = set()
    
    for question_id in question_ids:
        try:
            # Parse the question_id to extract components
            parts = question_id.split('_')
            if len(parts) >= 3:  # Ensure we have at least language, main_question_id, and sub_question_id
                language = parts[0]
                main_question_id = int(parts[1])
                
                # Create a unique identifier for this language-main_question_id pair
                pair_key = f"{language}_{main_question_id}"
                
                # Skip if we've already processed this pair
                if pair_key in processed_pairs:
                    continue
                
                processed_pairs.add(pair_key)
                
                logger.info(f"Filtering dataset for language: {language}, main_question_id: {main_question_id}")
                
                # Filter the dataset by both main_question_id and language
                current_filtered = dataset_df[
                    (dataset_df[DC.MAIN_QUESTION_ID] == main_question_id) & 
                    (dataset_df[DC.LANGUAGE] == language)
                ]
                
                if current_filtered.empty:
                    logger.warning(f"No matching questions found for {question_id}")
                else:
                    logger.info(f"Found {len(current_filtered)} questions matching {question_id}")
                    # Append to our results
                    filtered_results = pd.concat([filtered_results, current_filtered])
            else:
                logger.warning(f"Invalid question_id format: {question_id}")
        except Exception as e:
            logger.error(f"Error parsing question_id {question_id}: {str(e)}")
    
    # If we didn't find any matches, return the original dataset
    if filtered_results.empty:
        logger.warning("No matching questions found for any of the provided question_ids")
        return dataset_df, None
    logger.info(f"Total of {len(filtered_results)} questions matched across all filters")
    
    # Extract unique main_question_ids from filtered results
    filtered_main_question_ids = filtered_results[DC.MAIN_QUESTION_ID].unique().tolist()
    return filtered_results, filtered_main_question_ids


In [ ]:
dataset_df = load_dataset(config['data']['data_dir'])
dataset_df.head()

In [8]:
config['data']['question_id']

In [9]:
dataset_df, filtered_main_question_ids = filter_dataset_by_question_id(
        dataset_df, config['data']['question_id'])

In [ ]:
dataset_df.head()

In [ ]:
len(dataset_df) ,filtered_main_question_ids

## 4. Model Initialization

### 4.1 Setup the language model

In [12]:
# Setup the language model
def setup_model(model_config, generation_args):
    """Initialize and setup the language model."""
    # Convert generation_args to GenerationArguments if needed
    if isinstance(generation_args, dict):
        generation_config = GenerationArguments(**generation_args)
    else:
        generation_config = generation_args
        
    # Build the LLM model
    llm_config = {
        'model_config': model_config,
        'generation_config': generation_config
    }

    # If using custom model, load it from custom_llm.py
    if model_config.get("model_name") == "custom_llm":
        from custom_llm import MyCustomModel
        
    llm = LLM.build_from_config(llm_config)
    return llm


### 4.2 Setup OCR if needed

In [13]:
# Setup OCR if needed
def setup_ocr(use_ocr=False):
    """Setup OCR model if required."""
    ocr_model = None
    if use_ocr:
        try:
            from paddleocr import PaddleOCR
            ocr_model = PaddleOCR(use_angle_cls=True)
            logger.info("OCR model initialized successfully")
        except ImportError:
            logger.error("Failed to import PaddleOCR. Please install with: pip install paddleocr")
    return ocr_model

In [ ]:
generation_config = config.get("generation", {})
generation_config

In [ ]:
model_config = config["model"]
model_config

In [ ]:
llm = setup_model(model_config, generation_config)
llm

In [ ]:
llm_name = model_config.get("model_full_name", "unknown_model")
llm_name

In [18]:
use_ocr = model_config.get('use_ocr', False)
ocr_model = setup_ocr(use_ocr)
ocr_model

## 5. Setup Database

In [22]:
import dictdatabase as DDB
DDB.config.storage_directory = Path.joinpath(project_root, 'ddb_storage')
def initialize_database(output_db: str):
    """
    Initialize the database if it doesn't exist and return the database object.
    """
    db = DDB.at(output_db).read()
    if db is None:
        DDB.at(output_db).create()
        db = {}
        logger.info(f"Initialized new database: {output_db}")
    else:
        logger.info(f"Loaded existing database: {output_db} with {len(db)} entries.")
    return db

def setup_database(model_name, data_dir):
    """Initialize database for storing results."""
    release_version = data_dir.split('/')[-1].split('.')[0]
    target_db_name = f'{model_name}_ans_{release_version}'
    target_db = initialize_database(output_db=target_db_name)
    
    return target_db_name, target_db

def save_results(target_db_name, key, subquestion_responses):
    """Save responses to database."""
    write_to_database(target_db_name, key, subquestion_responses)

In [ ]:
config['data']['data_dir']

In [ ]:
target_db_name, target_db = setup_database(llm_name, config['data']['data_dir'])
target_db_name, target_db

## 6. Question Processing and Response Generation

In [25]:
def generate_answer_for_one_main_question(sub_question_set_df, llm, use_pot=False, 
                                          is_reasoning_model=False, use_ocr=False, 
                                          ocr_model=None, parent_dir=""):
    """Generate model responses for a set of related sub-questions."""
    # Get the context from the first sub_question
    context = sub_question_set_df.iloc[0].get("context", "")

    # Collect images from the first sub-question
    images = collect_images_from_first_subquestion(sub_question_set_df, parent_dir=parent_dir)

    # Format sub-questions
    sub_questions = []
    question_id_list = []
    for _, row in sub_question_set_df.iterrows():
        question_dict = {
            "id": row['question_id'],
            "type": row['question_type'],
            "question": row['question']
        }

        # Add options if it's a multiple-choice question
        if row['question_type'] == 'multiple-choice':
            question_dict["options"] = row['options']

        sub_questions.append(question_dict)
        question_id_list.append(row['question_id'])

    # Select prompt based on configuration
    if use_pot:
        prompt = ProgramOfThoughtsQuestionPrompt.init().format(
            context=context,
            sub_questions=sub_questions
        )
    else:
        prompt = QuestionPrompt.init().format(
            context=context,
            sub_questions=sub_questions
        )

    # Generate response
    model_output = generate_response_from_llm(llm, prompt, images, use_ocr=use_ocr, ocr_model=ocr_model)
    model_response = safe_parse_response(model_output, question_id_list, is_reasoning_model=is_reasoning_model)

    return model_response


## 7.Main Generation Process

In [25]:
# Get additional configuration parameters
use_pot = model_config.get('use_pot', False)
is_reasoning_model = model_config.get('is_reasoning_model', False)
# Create a copy of the DataFrame at the start
df_copy = dataset_df.copy()
    # Get parent directory for image paths
parent_dir = os.path.dirname(config['data']['data_dir'])


In [ ]:
# Process each main question group
for (_, language, main_question_id), group in df_copy.groupby(
        ['language_order', DC.LANGUAGE, DC.MAIN_QUESTION_ID]):
    
    key = f'{language}_{main_question_id}'
    
    # Skip if already in database AND not specifically requested in filtered_main_question_ids
    if key in target_db and (filtered_main_question_ids is None or 
                                main_question_id not in filtered_main_question_ids):
        logger.info(f'Skipping {key} - already in database and not specifically requested')
        continue
        
    try:
        logger.info(f'Start generating answers for {language} -- main_question_id: {main_question_id}')
        
        # Generate answers for this question group
        model_response = generate_answer_for_one_main_question(
            group, llm, use_pot=use_pot, is_reasoning_model=is_reasoning_model,
            use_ocr=use_ocr, ocr_model=ocr_model, parent_dir=parent_dir
        )
        
        # Aggregate all subquestions with their answers 
        subquestion_responses = {}
        for idx in range(len(group)):
            output_key = group.iloc[idx]['question_id']
            
            # Create a JSON object with the original input data and the model response
            input_data_with_response = group.iloc[idx].to_dict()
            
            if is_reasoning_model:
                input_data_with_response.update({
                    'model_answer': model_response[output_key].get('answer', ''),
                    'model_explanation': model_response[output_key].get('explanation', ''),
                    'model_reasoning': model_response.get('reasoning_content', '')
                })
            else:
                input_data_with_response.update({
                    'model_answer': model_response[output_key].get('answer', ''),
                    'model_explanation': model_response[output_key].get('explanation', '')
                })
            
            # Store the response in the subquestion_responses dictionary
            subquestion_responses[output_key] = input_data_with_response
        
        # Write the aggregated subquestion responses to the database
        save_results(target_db_name, key, subquestion_responses)
        logger.info(f'Successfully processed and saved {key}')
        
    except Exception as e:
        logger.error(f"Error processing main_question_id {main_question_id}: {str(e)}")
        continue
